In [2]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import locale
locale.setlocale(locale.LC_ALL, 'ID')
import re
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

In [ ]:
http://www.metrotvnews.com/index/2018/07/28
http://otomotif.metrotvnews.com/index/

## GET INDEKS LINK

In [424]:
def getIndeksLink(links, cat_link, date=datetime.strftime(datetime.today(), '%Y/%m/%d')):
    """
    Untuk mengambil seluruh url 
    link pada indeks category tertentu
    date format : YYYY-mm-dd
    """
#     print("page ", page)
    if cat_link == 'metrotvnews':
        url = "https://metrotvnews.com/index/"+date
    else :
        url = "https://"+cat_link+".metrotvnews.com/index/"+date
    print(url)
    # Make the request and create the response object: response
    response = requests.get(url)
    # Extract HTML texts contained in Response object: html
    html = response.text
    # Create a BeautifulSoup object from the HTML: soup
    soup = BeautifulSoup(html, "html5lib")
    contentDiv = soup.find('div', class_="style_06").findAll('h2')
    contentDiv
    for post in contentDiv:
        link = [post.find('a',href=True)['href']]
        links.append(link)
        
    el_page = soup.find('div', class_="extrapadding")
    if el_page:
        a_page = el_page.findAll('div', class_='bu fr')[-1].find('a')
        if  el_page.findAll('div', class_='paging__item')[-1].find('a', class_="paging__link paging__link--active"):
            max_page = page
        else:
            max_page = int(a_page['data-ci-pagination-page'].replace('\n', '').strip(' '))
            
        if page < max_page:
            links = getIndeksLink(links, page+1, cat_link, category, date)
        
    return links

In [203]:
soup.find('div', class_="extrapadding")

<div class="extrapadding">
    <div class="grid"><div class="bu fr"><a data-ci-pagination-page="2" href="http://news.metrotvnews.com/index/2018/07/27/30" rel="next">NEXT ›</a></div><div class="bu fr"><a data-ci-pagination-page="4" href="http://news.metrotvnews.com/index/2018/07/27/90">LAST »</a></div></div></div>

In [201]:
links = []
contentDiv = soup.find('div', class_="style_06").findAll('h2')
contentDiv
for post in contentDiv:
    link = [post.find('a',href=True)['href']]
    links.append(link)

In [202]:
links

[['http://news.metrotvnews.com/peristiwa/JKRn8m3K-staf-kjri-johor-bahru-belajar-jurnalistik-bersama-metro-tv'],
 ['http://news.metrotvnews.com/politik/eN4X9j7N-barisan-tgb-merapat-ke-nasdem'],
 ['http://news.metrotvnews.com/hukum/8Ky7DEOb-bupati-lampung-selatan-jadi-tersangka-suap'],
 ['http://news.metrotvnews.com/metro/aNrDzGVk-kasn-anies-langgar-prosedur-perombakan-pejabat-dki'],
 ['http://news.metrotvnews.com/politik/gNQngDwb-nasdem-coret-15-nama-bacaleg-eks-napi-korupsi'],
 ['http://news.metrotvnews.com/politik/4KZ4rY6b-petani-adukan-harga-gula-rendah-ke-ombudsman'],
 ['http://news.metrotvnews.com/peristiwa/RkjZn7Rk-asap-karhutla-dipastikan-tak-ganggu-asian-games'],
 ['http://news.metrotvnews.com/peristiwa/0kp21xnN-sumsel-tidak-liburkan-siswa-selama-asian-games'],
 ['http://news.metrotvnews.com/peristiwa/8N0V9mEk-menhut-tegaskan-isu-udara-jakarta-buruk-tidak-benar'],
 ['http://news.metrotvnews.com/politik/nN95ZzRN-pks-dinilai-berpotensi-hengkang-dari-poros-prabowo'],
 ['http://news

In [ ]:
el_page = soup.find('div', class_="paging__wrap clearfix")

In [155]:
url = "http://news.metrotvnews.com/index/2018/07/27"

In [128]:
# links = getIndeksLink([], 'otomotif')

## GET DETAIL BERITA

In [123]:
def getDetailBerita(url):
#     all_articles = []
#     for link in links:
    articles = {}
    #link
#         url = link[0]
    response = requests.get(url)
    html = response.text

    # Create a BeautifulSoup object from the HTML: soup
    soup = BeautifulSoup(html, "html5lib")

    #extract subcategory from breadcrumb
    bc = soup.find('div', class_="breadcrumbs")
#     if not bc:
#         continue

    sub = bc.findAll('a')[1].text
#     if ("foto" in sub.lower()) or  "video" in sub.lower(): 
#         continue

    articles['subcategory'] = sub

    #articles['id'] = int(soup.find("meta", attrs={'name':'articleid'})['content'])

    #category
#         articles['category'] = link[1]
    articles['url'] = url

    article = soup.find('div', class_="tru")

    #extract date
    pubdate_author = soup.find("div",class_="reg").text
    pubdate_author_split = pubdate_author.split(' \xa0\xa0 • \xa0\xa0 ')
    pubdate = pubdate_author_split[1]
    pubdate = pubdate.strip(' \t\n\r')
    pubdate = pubdate.replace(' WIB','')
    articles['pubdate'] = datetime.strftime(datetime.strptime(pubdate, "%A, %d %b %Y %H:%M"), "%Y-%m-%d %H:%M:%S")

    #extract author
    author = pubdate_author_split[0]
    articles['author'] = author

    #extract title
    articles['title'] = soup.find('meta', attrs={"property":"og:title"})['content']

    #source
    articles['source'] = 'metrotvnews'

    #extract comments count
    #articles['comments'] = int(soup.find('a', class_="komentar").find('span').text.replace('Komentar', '').strip(' \t\n\r'))
    articles['comment'] = print("?")
    
    #extract tags
    tags = soup.find('div', class_="line").findAll('a', class_="tag")
    articles['tags'] = ','.join([x.text for x in tags])

    #extract images
    articles['images'] = soup.find('img', class_="pic")['src']

    #extract detail
    detail = soup.find('div', class_="tru")

    #hapus link sisip
    for link in detail.findAll('div', class_="related"):
        link.decompose()

    #hapus video sisip
    for tag in detail.findAll('iframe', class_="embedv"):
        tag.decompose()

    #hapus all setelah clear fix
    #for det in detail.find('div', class_="wfull fl rl"):
    #    det.decompose()

    #hapus all script
    for script in detail.findAll('script'):
        script.decompose()

    #extract content
    detail = BeautifulSoup(detail.decode_contents().replace('<br/>', ' '), "html5lib")
    content = re.sub(r'\n|\t|\b|\r','',detail.text)
    articles['content'] = content.strip(' ')
    #print('memasukkan berita id ', articles['id'])
    #all_articles.append(articles)
    return articles

In [124]:
url = "http://news.metrotvnews.com/peristiwa/JKRn8m3K-staf-kjri-johor-bahru-belajar-jurnalistik-bersama-metro-tv"

In [156]:
response = requests.get(url)
    # Extract HTML texts contained in Response object: html
html = response.text
    # Create a BeautifulSoup object from the HTML: soup
soup = BeautifulSoup(html, "html5lib")

In [143]:
# articles['comments'] = int(soup.find('a', class_="komentar").find('span').text.replace('Komentar', '').strip(' \t\n\r'))
# articles['comments'] 

In [125]:
details = [getDetailBerita(url)]
details

?


[{'author': ' ',
  'comment': None,
  'content': 'Johor Bahru: Staf Konsulat Jenderal RI (KJRI) Johor Bahru, Malaysia, mendapatkan pengalaman belajar ilmu jurnalistik bersama Metro TV. Pelatihan ini dibuka oleh Konsul Jenderal RI di Johor Bahru, Haris Nugroho.  "Sayang kalau pelatihan ini tidak dimanfaatkan dengan baik. Apalagi kita sudah mengundang narasumber dari Metro TV yang didatangkan langsung dari Jakarta," kata Haris saat memberi sambutan di depan 50 peserta pelatihan, di kantor perwakilan KJRI di Johor Bahru, Malaysia, Jumat, 27 Juli 2018.     Haris mengingatkan seluruh peserta agar mengaplikasikan pengetahuan yang diperoleh untuk meningkatkan kualitas karya jurnalistik, khususnya di unit penerangan KJRI Johor Bahru. Ia juga berharap pelatihan ini bisa meningkatkan pengetahuan peserta.  "Usai pelatihan ini saya berharap peserta mampu menunjukkan kualitas kerja dan fungsi pelayanan informasi di lingkungan kantor perwakilan kita," kata Haris.  Bertindak sebagai narasumber pelati

In [126]:
df = pd.DataFrame(details)

In [127]:
df

,author,comment,content,images,pubdate,source,subcategory,tags,title,url
0,,None,Johor Bahru: Staf Konsulat Jenderal RI (KJRI) ...,http://cdn.metrotvnews.com/dynamic/content/201...,2018-07-27 23:19:00,metrotvnews,Peristiwa,"metro tv,jurnalistik",Staf KJRI Johor Bahru Belajar Jurnalistik Bers...,http://news.metrotvnews.com/peristiwa/JKRn8m3K...


In [122]:
# options = Options()
# options.add_argument('--headless')
# options.add_argument('--disable-gpu')  # Last I checked this was necessary.
# options.add_argument('--disable-extensions')

# driver = webdriver.Chrome("../chromedriver.exe", chrome_options=options)
# driver.get(url)
# html = driver.page_source
# soup = BeautifulSoup(html, 'html5lib')

# comment = soup.find('div', class_="span4 comments-count tright")
# comment_num = comment.text.replace('Ada ', '')
# comment_num = comment_num.replace(' komentar untuk artikel ini', '')